In [2]:
# Import necessary libraries
import pandas as pd
import json
import sqlite3

# --- Step 1: Load CSV Data ---
orders_df = pd.read_csv('orders.csv')

# --- Step 2: Load JSON Data ---
with open('users.json', 'r') as f:
    users_data = json.load(f)
users_df = pd.DataFrame(users_data)

# --- Step 3: Load SQL Data ---
# Using sqlite3 to create an in-memory database to parse the SQL file
conn = sqlite3.connect(':memory:')
with open('restaurants.sql', 'r') as f:
    sql_script = f.read()
conn.executescript(sql_script)
restaurants_df = pd.read_sql_query("SELECT * FROM restaurants", conn)
conn.close()

# --- Step 4: Merge the Data ---
# Merge orders with users (Left Join)
merged_df = orders_df.merge(users_df, on='user_id', how='left')

# Merge with restaurants (Left Join)
# Using suffixes to differentiate duplicate columns like 'restaurant_name'
final_df = merged_df.merge(restaurants_df, on='restaurant_id', how='left', suffixes=('_order', '_rest'))

# --- Step 5: Create Final Dataset ---
final_df.to_csv('final_food_delivery_dataset.csv', index=False)
print("Final dataset 'final_food_delivery_dataset.csv' created successfully.")

# --- DATA ANALYSIS SECTION ---

# 1. Total Revenue from Gold members by City
gold_rev_city = final_df[final_df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().sort_values(ascending=False)
print("\nRevenue from Gold members by City:\n", gold_rev_city)

# 2. Average Order Value by Cuisine
avg_order_cuisine = final_df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)
print("\nAverage Order Value by Cuisine:\n", avg_order_cuisine)

# 3. Users with total spending > ₹1000
high_spenders = (final_df.groupby('user_id')['total_amount'].sum() > 1000).sum()
print(f"\nDistinct users with > ₹1000 total spend: {high_spenders}")

# 4. Revenue by Restaurant Rating Range
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0 – 3.5', '3.6 – 4.0', '4.1 – 4.5', '4.6 – 5.0']
final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels, include_lowest=True)
rating_rev = final_df.groupby('rating_range', observed=False)['total_amount'].sum().sort_values(ascending=False)
print("\nRevenue by Rating Range:\n", rating_rev)

# 5. Average Order Value for Gold members by City
gold_avg_city = final_df[final_df['membership'] == 'Gold'].groupby('city')['total_amount'].mean().sort_values(ascending=False)
print("\nAvg Order Value from Gold members by City:\n", gold_avg_city)

# 6. Percentage of total orders by Gold members
gold_orders_pct = (len(final_df[final_df['membership'] == 'Gold']) / len(final_df)) * 100
print(f"\nPercentage of orders by Gold members: {round(gold_orders_pct)}%")

# 7. Quarterly Revenue Distribution
final_df['order_date'] = pd.to_datetime(final_df['order_date'], dayfirst=True)
final_df['quarter'] = final_df['order_date'].dt.quarter
quarterly_rev = final_df.groupby('quarter')['total_amount'].sum().sort_values(ascending=False)
print("\nTotal Revenue by Quarter:\n", quarterly_rev)

Final dataset 'final_food_delivery_dataset.csv' created successfully.

Revenue from Gold members by City:
 city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

Average Order Value by Cuisine:
 cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

Distinct users with > ₹1000 total spend: 2544

Revenue by Rating Range:
 rating_range
4.6 – 5.0    2197030.75
3.0 – 3.5    2136772.70
4.1 – 4.5    1960326.26
3.6 – 4.0    1717494.41
Name: total_amount, dtype: float64

Avg Order Value from Gold members by City:
 city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

Percentage of orders by Gold members: 50%

Total Revenue by Quarter:
 quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64
